In [1]:
import pandas as pd
import os 
import numpy as np

ruta = os.path.abspath('../Datasets')

In [15]:
df_venta = pd.read_csv(ruta+'/Venta.csv', sep=',', encoding='utf-8')

In [7]:
df_venta.head(3)

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2.0
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3.0
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1.0


Vemos que la tabla de venta tiene una estructura similar a la de compras, usaremos el mismo análisis por ahora

## Filas Duplicadas

In [20]:
#eliminando IdVenta
df_venta.drop(['IdVenta'],axis=1,inplace=True)
#filas duplicadas
df_venta.duplicated().sum()

15

In [28]:
df_venta.shape[0]

46180

In [29]:
df_venta.drop_duplicates(inplace=True)

In [30]:
df_venta.shape[0]

46165

## Detección de Outliers

In [149]:
Q1 = df_venta['Precio'].quantile(.25)
Q3 = df_venta['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

In [150]:
ubicacion_outliers = (df_venta.Precio < minimo) | (df_venta.Precio > maximo)
df_venta[ubicacion_outliers].sort_values('Precio')

,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
11,2015-02-17,2015-02-18,1,2866,13,1674,42969,3839.0,2.0
20848,2016-03-07,2016-03-12,3,1855,10,2468,42969,3839.0,3.0
20842,2015-09-15,2015-09-20,1,1463,10,2468,42969,3839.0,2.0
20662,2017-06-01,2017-06-04,3,2650,10,2468,42969,3839.0,1.0
20557,2016-12-27,2017-01-04,2,2017,10,2779,42969,3839.0,2.0
...,...,...,...,...,...,...,...,...,...
9143,2017-02-01,2017-02-08,1,1469,3,1012,42987,4560000.0,1.0
43564,2020-07-27,2020-08-01,2,1704,14,1531,42949,20150000.0,1.0
42053,2018-10-08,2018-10-11,2,636,31,3043,42937,24580000.0,3.0
20904,2015-07-15,2015-07-16,3,144,10,2351,42811,25599200.0,2.0


In [153]:
df_id = df_venta[df_venta.IdProducto == 42969]
Q1 = df_id['Precio'].quantile(.25)
Q3 = df_id['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

In [156]:
ubicacion_outliers = (df_id.Precio < minimo) | (df_id.Precio > maximo)
df_id[ubicacion_outliers]

,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
5238,2020-03-31,2020-04-05,2,3402,2,1676,42969,383900.0,2.0
11672,2018-11-23,2018-11-27,1,1991,5,1572,42969,383900.0,1.0
29812,2016-11-25,2016-11-30,2,24,21,3766,42969,383900.0,2.0
34871,2019-07-22,2019-07-30,2,2884,25,3916,42969,383900.0,3.0


In [157]:
#calculamos la media, descartando los outliers
outlier = list(df_id[ubicacion_outliers].Precio)
media = (df_id.Precio.sum() - sum(outlier)) / (len(df_id) - len(outlier))
media

3766.1075949367087

proceso de busqueda de unión df_venta y ubicación_outliers ...

In [159]:
#desempacantdo mi variable ubicación outliers
list(zip(*np.where(ubicacion_outliers)))

[(26,), (41,), (110,), (118,)]

In [160]:
#desempaquetando los indices a imputar
outlier = []
for i in list(zip(*np.where(ubicacion_outliers))):
    ii = i[0]
    outlier.append(ii)
outlier

[26, 41, 110, 118]

no coinciden con los indices que deseo imputar...pero esto me da una idea...

In [148]:
df_venta.loc[list(df_id[ubicacion_outliers].index)]

,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
5238,2020-03-31,2020-04-05,2,3402,2,1676,42969,383900.0,2.0
11672,2018-11-23,2018-11-27,1,1991,5,1572,42969,383900.0,1.0
29812,2016-11-25,2016-11-30,2,24,21,3766,42969,383900.0,2.0
34871,2019-07-22,2019-07-30,2,2884,25,3916,42969,383900.0,3.0


ahora ya podemos reemplazar los valores atipicos por la media en el dataframe df_venta !!!

In [161]:
#reemplazando valores atipicos en el dataframe principal
df_venta.loc[list(df_id[ubicacion_outliers].index)] = round(media, 2)

In [172]:
df_venta.loc[df_venta.index == 5238]

,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
5238,3766.11,3766.11,3766.11,3766.11,3766.11,3766.11,3766.11,3766.11,3766.11


Verificamos si los outliers fueron reemplazados

In [175]:
df_id = df_venta[df_venta.IdProducto == 42969]
Q1 = df_id['Precio'].quantile(.25)
Q3 = df_id['Precio'].quantile(.75)
IQR = Q3 -Q1
minimo = Q1 - 1.5 * IQR
maximo = Q3 + 1.5 * IQR

ubicacion_outliers = (df_id.Precio < minimo) | (df_id.Precio > maximo)
df_id[ubicacion_outliers]

,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad


## Detección de Valores Nulos

In [177]:
#capturando los valores nulos por columna
def valor_nulo(e):
    diccionario = {}
    for column in e.columns:
        val_nul = e[column].isna().sum()
        diccionario[column] = int(val_nul)

    return diccionario  

In [178]:
nulo = valor_nulo(df_venta)
nulo = [[key, nulo[key]] for key in nulo.keys() if nulo[key] > 0]
nulo

[['Precio', 920], ['Cantidad', 884]]

En conclusión, podemos observar que la estructura es similiar al archivo Compra.csv , con lo cual trataremos estas dos tablas de manera similiar al transformar sus datos.